In [9]:
!pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jangedoo/utkface-new")

print("Path to dataset files:", path)

Path to dataset files: /Users/teodimov/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1


In [12]:
from glob import glob

image_files = glob(os.path.join(dataset_path, '**/*.*'), recursive=True)
print("Number of files found:", len(image_files))
print("Sample files:", image_files[:5])

Number of files found: 66976
Sample files: ['/Users/teodimov/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1/crop_part1/9_1_2_20161219204347420.jpg.chip.jpg', '/Users/teodimov/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1/crop_part1/1_1_2_20161219154612988.jpg.chip.jpg', '/Users/teodimov/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1/crop_part1/16_0_0_20170104003740977.jpg.chip.jpg', '/Users/teodimov/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1/crop_part1/1_1_3_20161219230734016.jpg.chip.jpg', '/Users/teodimov/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1/crop_part1/26_1_1_20170103181931657.jpg.chip.jpg']


In [38]:
%conda create -n vit_env python=3.9 -y
%conda activate vit_env
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install transformers

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
| DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/osx-arm64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/osx-arm64/current_repodata.json HTTP/1.1" 304 0
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.2
  latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated 

In [16]:
from glob import glob

class UTKFaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = glob(os.path.join(root_dir, '**/*.chip.jpg'), recursive=True)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        image = Image.open(img_name).convert('RGB')
        label = int(os.path.basename(img_name).split('_')[0])  # Extract age

        if self.transform:
            image = self.transform(image)

        return image, label

In [17]:
dataset_path = "/Users/teodimov/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1"
dataset = UTKFaceDataset(root_dir=dataset_path, transform=transform)

print("Number of samples in dataset:", len(dataset))

# Check a sample
sample_image, sample_label = dataset[0]
print("Sample image shape:", sample_image.shape)
print("Sample label (age):", sample_label)

Number of samples in dataset: 66976
Sample image shape: torch.Size([3, 224, 224])
Sample label (age): 9


In [18]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

dataset_path = "/Users/teodimov/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1"
dataset = UTKFaceDataset(root_dir=dataset_path, transform=transform)

# Split into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [15]:
from transformers import ViTForImageClassification
import torch.nn as nn

# Load pre-trained Vision Transformer
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k")

# Modify the classification head for regression
model.classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, 256),  # Add a hidden layer
    nn.ReLU(),
    nn.Linear(256, 1)  # Output a single value for regression
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/Users/teodimov/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

# Training loop
epochs = 5  # Adjust based on your preference
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images = images.to(device)
        labels = labels.to(device, dtype=torch.float32).unsqueeze(1)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {running_loss / len(train_loader):.4f}")


Epoch 1/5:   0%|          | 2/1675 [00:46<10:47:41, 23.23s/it]


KeyboardInterrupt: 

In [ ]:
# Evaluate model
model.eval()
val_labels = []
val_predictions = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device, dtype=torch.float32)

        outputs = model(images).logits.squeeze()
        val_labels.extend(labels.cpu().numpy())
        val_predictions.extend(outputs.cpu().numpy())

mae = mean_absolute_error(val_labels, val_predictions)
print(f"Validation Mean Absolute Error (MAE): {mae:.2f}")

In [ ]:
torch.save(model.state_dict(), "vit_age_prediction.pth")
print("Model saved as vit_age_prediction.pth")